# Encargo 3
### María José Namuche Bustamante

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense,LSTM

## Carga de datos

In [2]:
data = pd.read_csv("pacientes_diabetes.csv")
data.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


Se observa que no se cuenta con encabezado y que la primera fila se ha tomado como nombre de la columna por lo que se estaría perdiendo una fila de información si solo se renombrara las columnas.

In [3]:
data.shape

(767, 9)

In [4]:
data.columns

Index(['6', '148', '72', '35', '0', '33.6', '0.627', '50', '1'], dtype='object')

In [5]:
new = [(data.columns)]
new
df=pd.DataFrame(new, columns = ['Num_emb', 'Conc_gluc', 'Pres', 'Grosor', 'Insulina', 'IMC', 'Func_Pdg', 'Edad', 'Clase'])
df

,Num_emb,Conc_gluc,Pres,Grosor,Insulina,IMC,Func_Pdg,Edad,Clase
0,6,148,72,35,0,33.6,0.627,50,1


In [6]:
data.columns = df.columns

In [7]:
df = df.append(data,ignore_index=True)
df

,Num_emb,Conc_gluc,Pres,Grosor,Insulina,IMC,Func_Pdg,Edad,Clase
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.34,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [8]:
df['Num_emb'] = pd.to_numeric(df['Num_emb'])
df['Conc_gluc'] = pd.to_numeric(df['Conc_gluc'])
df['Pres'] = pd.to_numeric(df['Pres'])
df['Grosor'] = pd.to_numeric(df['Grosor'])
df['Insulina'] = pd.to_numeric(df['Insulina'])
df['IMC'] = pd.to_numeric(df['IMC'])
df['Func_Pdg'] = pd.to_numeric(df['Func_Pdg'])
df['Edad'] = pd.to_numeric(df['Edad'])
df['Clase'] = pd.to_numeric(df['Clase'])

In [9]:
df.dtypes

Num_emb        int64
Conc_gluc      int64
Pres           int64
Grosor         int64
Insulina       int64
IMC          float64
Func_Pdg     float64
Edad           int64
Clase          int64
dtype: object

In [10]:
# Datos nulos
df.isnull().sum()

Num_emb      0
Conc_gluc    0
Pres         0
Grosor       0
Insulina     0
IMC          0
Func_Pdg     0
Edad         0
Clase        0
dtype: int64

Se puede apreciar que no hay ningún dato nulo

In [11]:
df.describe()

,Num_emb,Conc_gluc,Pres,Grosor,Insulina,IMC,Func_Pdg,Edad,Clase
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_emb    768 non-null    int64  
 1   Conc_gluc  768 non-null    int64  
 2   Pres       768 non-null    int64  
 3   Grosor     768 non-null    int64  
 4   Insulina   768 non-null    int64  
 5   IMC        768 non-null    float64
 6   Func_Pdg   768 non-null    float64
 7   Edad       768 non-null    int64  
 8   Clase      768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [13]:
print(df.groupby("Clase").size())

Clase
0    500
1    268
dtype: int64


## Data de entrenamiento

In [14]:
target = pd.DataFrame(df['Clase'])
target

,Clase
0,1
1,0
2,1
3,0
4,1
...,...
763,0
764,0
765,0
766,1


In [15]:
target = np.array(target, 'int64')
target

array([[1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
    

In [16]:
training = df.drop(['Clase'], axis=1)
training

,Num_emb,Conc_gluc,Pres,Grosor,Insulina,IMC,Func_Pdg,Edad
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [17]:
training = np.array(training)

In [18]:
training

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

# Arquitectura del modelo

In [19]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Se crea un modelo vacio de tipo Sequential, pues crearemos una serie de capas de neuronas secuenciales.
Se tendrán 8 entradas y en la capa oculta 12 neuronas. Además se usa como función de activación a "relu". También se agrega una capa con 1 neurona de salida y función de activación sigmoid, ya que tenemos la intención de generar una aproximación del gradiente igual a 0.

## Modelo de compilación

In [20]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Modelo de ajuste

In [21]:
model.fit(training, target, epochs=1000)

Epoch 1/1000
24/24 [==============================] - 1s 1ms/step - loss: 12.0091 - accuracy: 0.5768
Epoch 2/1000
24/24 [==============================] - 0s 1ms/step - loss: 6.8197 - accuracy: 0.5599
Epoch 3/1000
24/24 [==============================] - 0s 1ms/step - loss: 3.3091 - accuracy: 0.5208
Epoch 4/1000
24/24 [==============================] - 0s 1ms/step - loss: 1.8703 - accuracy: 0.5638
Epoch 5/1000
24/24 [==============================] - 0s 2ms/step - loss: 1.3962 - accuracy: 0.6250
Epoch 6/1000
24/24 [==============================] - 0s 2ms/step - loss: 1.2067 - accuracy: 0.6484
Epoch 7/1000
24/24 [==============================] - 0s 1ms/step - loss: 1.1062 - accuracy: 0.6432
Epoch 8/1000
24/24 [==============================] - 0s 2ms/step - loss: 1.0502 - accuracy: 0.6406
Epoch 9/1000
24/24 [==============================] - 0s 1ms/step - loss: 1.0128 - accuracy: 0.6484
Epoch 10/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.9772 - accuracy: 0.660

24/24 [==============================] - 0s 1ms/step - loss: 0.5991 - accuracy: 0.6862
Epoch 82/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.5999 - accuracy: 0.6927
Epoch 83/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.6014 - accuracy: 0.6849
Epoch 84/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5950 - accuracy: 0.6992
Epoch 85/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5938 - accuracy: 0.6953
Epoch 86/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5941 - accuracy: 0.6888
Epoch 87/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5913 - accuracy: 0.6979
Epoch 88/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5961 - accuracy: 0.6914
Epoch 89/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.5925 - accuracy: 0.6914
Epoch 90/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5901 - accuracy: 0.7031
Epoc

24/24 [==============================] - 0s 1ms/step - loss: 0.5362 - accuracy: 0.7383
Epoch 162/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5352 - accuracy: 0.7357
Epoch 163/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5327 - accuracy: 0.7474
Epoch 164/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5355 - accuracy: 0.7292
Epoch 165/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5287 - accuracy: 0.7461
Epoch 166/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5288 - accuracy: 0.7370
Epoch 167/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5314 - accuracy: 0.7305
Epoch 168/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.5305 - accuracy: 0.7409
Epoch 169/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5295 - accuracy: 0.7409
Epoch 170/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.5304 - accuracy: 0.

24/24 [==============================] - 0s 1ms/step - loss: 0.5140 - accuracy: 0.7461
Epoch 242/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.5114 - accuracy: 0.7448
Epoch 243/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5098 - accuracy: 0.7526
Epoch 244/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.5071 - accuracy: 0.7539
Epoch 245/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5103 - accuracy: 0.7422
Epoch 246/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.5059 - accuracy: 0.7565
Epoch 247/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5056 - accuracy: 0.7487
Epoch 248/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5049 - accuracy: 0.7461
Epoch 249/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5148 - accuracy: 0.7500
Epoch 250/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5078 - accuracy: 0.

24/24 [==============================] - 0s 1ms/step - loss: 0.4927 - accuracy: 0.7435
Epoch 322/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4886 - accuracy: 0.7643
Epoch 323/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4889 - accuracy: 0.7513
Epoch 324/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4924 - accuracy: 0.7500
Epoch 325/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4885 - accuracy: 0.7643
Epoch 326/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4964 - accuracy: 0.7487
Epoch 327/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4876 - accuracy: 0.7578
Epoch 328/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4935 - accuracy: 0.7539
Epoch 329/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4859 - accuracy: 0.7695
Epoch 330/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4903 - accuracy: 0.

...

24/24 [==============================] - 0s 1ms/step - loss: 0.4335 - accuracy: 0.7904
Epoch 640/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.7878
Epoch 641/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4297 - accuracy: 0.7917
Epoch 642/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4358 - accuracy: 0.7917
Epoch 643/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4347 - accuracy: 0.7826
Epoch 644/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4271 - accuracy: 0.7982
Epoch 645/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4288 - accuracy: 0.8047
Epoch 646/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4288 - accuracy: 0.7943
Epoch 647/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.8021
Epoch 648/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4272 - accuracy: 0.

24/24 [==============================] - 0s 1ms/step - loss: 0.4248 - accuracy: 0.7969
Epoch 719/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4215 - accuracy: 0.8034
Epoch 720/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4196 - accuracy: 0.8073
Epoch 721/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4166 - accuracy: 0.7982
Epoch 722/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4177 - accuracy: 0.8073
Epoch 723/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4310 - accuracy: 0.8034
Epoch 724/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.7956
Epoch 725/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4218 - accuracy: 0.7969
Epoch 726/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4267 - accuracy: 0.7878
Epoch 727/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4349 - accuracy: 0.

Epoch 798/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4162 - accuracy: 0.8021
Epoch 799/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4111 - accuracy: 0.8190
Epoch 800/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4089 - accuracy: 0.8099
Epoch 801/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4062 - accuracy: 0.8216
Epoch 802/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4107 - accuracy: 0.8164
Epoch 803/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4194 - accuracy: 0.8086
Epoch 804/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4089 - accuracy: 0.8125
Epoch 805/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4089 - accuracy: 0.8112
Epoch 806/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4102 - accuracy: 0.8125
Epoch 807/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4215

24/24 [==============================] - 0s 1ms/step - loss: 0.4039 - accuracy: 0.8138
Epoch 878/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4026 - accuracy: 0.8190
Epoch 879/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4157 - accuracy: 0.8151
Epoch 880/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4069 - accuracy: 0.8255
Epoch 881/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4078 - accuracy: 0.8164
Epoch 882/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4041 - accuracy: 0.8125
Epoch 883/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4098 - accuracy: 0.8125
Epoch 884/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4075 - accuracy: 0.8073
Epoch 885/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4005 - accuracy: 0.8177
Epoch 886/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4070 - accuracy: 0.

24/24 [==============================] - 0s 1ms/step - loss: 0.3976 - accuracy: 0.8203
Epoch 958/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4086 - accuracy: 0.8177
Epoch 959/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4044 - accuracy: 0.8203
Epoch 960/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4037 - accuracy: 0.8151
Epoch 961/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.3983 - accuracy: 0.8190
Epoch 962/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.3985 - accuracy: 0.8229
Epoch 963/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4013 - accuracy: 0.8138
Epoch 964/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4028 - accuracy: 0.8138
Epoch 965/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4070 - accuracy: 0.8164
Epoch 966/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.4074 - accuracy: 0.

## Evaluación del modelo

In [22]:
scores = model.evaluate(training, target)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

24/24 [==============================] - 0s 2ms/step - loss: 0.3895 - accuracy: 0.8281

accuracy: 82.81%


Se obtuvo un 82.81% de precisión

## Predicción del modelo
Utilizando la misma data

In [23]:
predictions = model.predict(training)
rounded = [round(x[0]) for x in predictions]
print(rounded)

[1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 

In [24]:
dataframe = pd.DataFrame(df['Clase'])

In [25]:
pred = pd.DataFrame(rounded, columns = ['Pred'])

In [26]:
dataframe['Pred'] = pred['Pred']

In [27]:
dataframe['Clase'] = pd.to_numeric(dataframe['Clase'])
dataframe['Pred'] = pd.to_numeric(dataframe['Pred'])

In [28]:
dataframe['Suma'] = dataframe['Clase'] + dataframe['Pred']

In [29]:
print(dataframe.groupby("Suma").size())

Suma
0    444
1    132
2    192
dtype: int64


Los valores 1 en la columna Suma, indican que la predicción fue incorrecta por lo que comprobamos que se cuenta con un 82.81% de precisión.